In [ ]:
import pygame
import numpy as np
import random
import time

COLOR_BG = (10, 10, 10) 
COLOR_GRID = (40, 40, 40) 
COLOR_DIE_NEXT = (170, 170, 170) 
COLOR_ALIVE_NEXT = (255, 255, 255)

#ASSIGNEMENTS
PATTERNS = {
    "block": np.array([[1,1], [1,1]]),  #Still Life
    "blinker": np.array([[1,1,1]]),  #Oscillator
    "glider": np.array([[0,0,1], [1,0,1], [0,1,1]]),  #Spaceship
    "eater_one": np.array([[1,1,0,0], [1,0,1,0], [0,0,1,0], [0,0,1,1]]), #Still Life
    "herschel": np.array([[1,0,0], [1,1,1], [1,0,1], [0,0,1]]), #Methuselah (Advanced pattern)
    "switch_engine": np.array([[0,1,0,1,0,0], [1,0,0,0,0,0], [0,1,0,0,1,0], [0,0,0,1,1,1]]), #Methuselah (Advanced pattern)
    "gosper_glider_gun": np.array([
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]) #Gun (Advanced pattern)
}

def update(screen, cells, size, with_progress=False):
    update_cells = np.zeros(cells.shape, dtype=int)
    rows, cols = cells.shape
    
    for row, col in np.ndindex(cells.shape):
        alive = sum(
            cells[(row + i) % rows, (col + j) % cols]
            for i in [-1, 0, 1]
            for j in [-1, 0, 1]
            if not (i == 0 and j == 0)
        )
        
        color = COLOR_BG if cells[row, col] == 0 else COLOR_ALIVE_NEXT
        
        if cells[row, col] == 1:
            if alive < 2 or alive > 3:
                if with_progress:
                    color = COLOR_DIE_NEXT
            elif 2 <= alive <= 3:
                update_cells[row, col] = 1
                if with_progress:
                    color = COLOR_ALIVE_NEXT
        else:
            if alive == 3:
                update_cells[row, col] = 1
                if with_progress:
                    color = COLOR_ALIVE_NEXT
        
        pygame.draw.rect(screen, color, (col * size, row * size, size - 1, size - 1))
    
    return update_cells

def reset(cells, rows, cols, sigma=5):
    m = 3  # Numero di centri gaussiani
    centers = np.column_stack((np.random.randint(0, rows, size=m), np.random.randint(0, cols, size=m)))
    prob_grid = np.zeros(cells.shape)
    for center in centers:
        distance_squared = (np.indices(cells.shape)[0] - center[0])**2 + (np.indices(cells.shape)[1] - center[1])**2
        prob_grid += np.exp(-distance_squared / (2 * sigma**2))
    prob_grid /= prob_grid.max()
    cells[:, :] = np.random.rand(rows, cols) < prob_grid
    return cells  # Aggiunto il return della matrice aggiornata



def generate_random(cells):
    cells[:, :] = np.random.randint(0, 20, size=cells.shape)
    return cells

def insert_pattern(cells, pattern_name, x, y):
    pattern = PATTERNS.get(pattern_name)
    if pattern is not None:
        rows, cols = pattern.shape
        cells[x:x+rows, y:y+cols] = pattern
    return cells

def main():
    pygame.init()
    screen = pygame.display.set_mode((1200, 600))
    clock = pygame.time.Clock()
    cells = np.zeros((80, 120))  
    size = 10  
    screen.fill(COLOR_GRID)
    rows, cols = 80, 120
    cells = np.zeros((rows, cols), dtype=int)
    reset(cells, rows, cols)  # Reset con la nuova funzionalità gaussiana
    update(screen, cells, size)
    pygame.display.flip()
    
    running = False
    drawing = False
    selected_pattern = "glider"  

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    running = not running
                elif event.key == pygame.K_q: 
                    cells = reset(cells, rows, cols)  # La funzione reset restituisce ora la matrice aggiornata

                elif event.key == pygame.K_g:  
                    cells = generate_random(cells)
                    update(screen, cells, size)
                    pygame.display.update()
                elif event.key == pygame.K_1:
                    selected_pattern = "block" #key1 to print block 
                elif event.key == pygame.K_2:
                    selected_pattern = "blinker" #key2 to print blinker 
                elif event.key == pygame.K_3:
                    selected_pattern = "glider" #key3 to print glider 
                elif event.key == pygame.K_4:
                    selected_pattern = "eater_one" #key4 to print eater 1 
                elif event.key == pygame.K_5:
                    selected_pattern = "herschel" #key5 to print herschel 
                elif event.key == pygame.K_6:
                    selected_pattern = "switch_engine" #key6 to print switch engine 
                elif event.key == pygame.K_7:
                    selected_pattern = "gosper_glider_gun" #key7 to print gosper glider gun 
            elif event.type == pygame.MOUSEBUTTONDOWN:
                pos = pygame.mouse.get_pos()
                row,col= pos[1]// size, pos[0]//size  #Convert pixels coordinates to grid index  
                if event.button == 1:
                    drawing = True
                    cells[row, col] = 1
                elif event.button == 3:  #Right-click=insert the selected pattern
                    cells = insert_pattern(cells, selected_pattern, row, col)
            elif event.type == pygame.MOUSEBUTTONUP and event.button == 1:
                drawing = False
            elif event.type == pygame.MOUSEMOTION and drawing:
                pos = pygame.mouse.get_pos()
                row,col =pos[1]//size,pos[0]//size
                cells[row,col] = 1  

        screen.fill(COLOR_GRID)
        if running:
            cells = update(screen, cells, size, with_progress=True)
        else:
            update(screen, cells, size)

        pygame.display.update()
        clock.tick(10)
        time.sleep(0.001)

if __name__ == '__main__':
    main()

pygame-ce 2.4.1 (SDL 2.28.4, Python 3.12.7)


In [ ]:
import pygame
import numpy as np
import random
import time
import matplotlib.pyplot as plt

def update(cells):
    update_cells = np.zeros(cells.shape, dtype=int)
    rows, cols = cells.shape
    for row, col in np.ndindex(cells.shape):
        alive = sum(
            cells[(row + i) % rows, (col + j) % cols]
            for i in [-1, 0, 1]
            for j in [-1, 0, 1]
            if not (i == 0 and j == 0)
        )
        if cells[row, col] == 1:
            if alive < 2 or alive > 3:
                update_cells[row, col] = 0
            elif 2 <= alive <= 3:
                update_cells[row, col] = 1
        elif alive == 3:
            update_cells[row, col] = 1
    return update_cells

def simulate_iterations(rows, cols, iterations=1000, max_steps=100):
    all_live_counts = []
    
    for _ in range(iterations):
        # Inizializza la griglia
        cells = np.zeros((rows, cols), dtype=int)
        generate_random(cells)
        
        live_counts = []
        for step in range(max_steps):
            cells = update(cells)
            live_count = np.sum(cells)  # Conta le celle vive
            live_counts.append(live_count)
        
        all_live_counts.append(live_counts)
    
    # Calcolare la media e la deviazione standard
    all_live_counts = np.array(all_live_counts)
    mean_live_counts = np.mean(all_live_counts, axis=0)
    std_live_counts = np.std(all_live_counts, axis=0)
    
    return mean_live_counts, std_live_counts
def generate_random(cells):
    cells[:, :] = np.random.randint(0, 2, size=cells.shape)
    return cells



def plot_live_cells(mean_live_counts, std_live_counts, total_cells):
    # Calcolare la percentuale di celle vive rispetto al totale delle celle
    mean_live_percent = (mean_live_counts / total_cells) * 100
    std_live_percent = (std_live_counts / total_cells) * 100

    # Iterazioni (a partire dall'iterazione 2)
    iterations = np.arange(2, len(mean_live_percent) + 1)

    plt.figure(figsize=(12, 6))  # Dimensione più grande per un grafico professionale
    plt.plot(iterations, mean_live_percent[1:], label="Media celle vive (%)", color="b", linewidth=2)
    plt.fill_between(iterations, 
                     mean_live_percent[1:] - std_live_percent[1:], 
                     mean_live_percent[1:] + std_live_percent[1:], 
                     color="b", alpha=0.3, label="Intervallo di incertezza")

    # Miglioramenti estetici
    plt.title('Evolution of the percentage of live cells in Game of Life', fontsize=16, fontweight='bold')
    plt.xlabel('Iteration', fontsize=14)
    plt.ylabel('Alive cells (%)', fontsize=14)

    # Legenda con bordi spessi
    legend = plt.legend(frameon=True, fontsize=12, borderpad=1, loc='upper right', fancybox=True, framealpha=1, edgecolor='black')
    legend.get_frame().set_linewidth(2)  # Imposta lo spessore del bordo della legenda

    # Rimuovere la griglia
    plt.grid(False)

    # Aggiungere margini per un aspetto migliore
    plt.tight_layout()

    # Visualizzare il grafico
    plt.show()




# Simula e visualizza i risultati
rows, cols = 50, 50  # Dimensioni della griglia
iterations = 5  # Numero di simulazioni
max_steps = 500  # Numero massimo di passi per simulazione
total_cells= 9600
mean_live_counts, std_live_counts = simulate_iterations(rows, cols, iterations, max_steps)
plot_live_cells(mean_live_counts, std_live_counts, total_cells)

In [ ]:
def detect_pattern(cells, pattern):
    rows, cols = cells.shape
    pattern_rows, pattern_cols = pattern.shape
    occurrences = []

    for row in range(rows - pattern_rows + 1):
        for col in range(cols - pattern_cols + 1):
            if np.array_equal(cells[row:row+pattern_rows, col:col+pattern_cols], pattern):
                occurrences.append((row, col))
    
    return occurrences

def track_pattern_frequency(cells, patterns, step):
    pattern_frequencies = {key: 0 for key in patterns.keys()}
    
    for pattern_name, pattern in patterns.items():
        occurrences = detect_pattern(cells, pattern)
        pattern_frequencies[pattern_name] = len(occurrences)

    return pattern_frequencies

def track_replication(cells, previous_cells, patterns):
    replication_count = {key: 0 for key in patterns.keys()}

    for pattern_name, pattern in patterns.items():
        occurrences_current = detect_pattern(cells, pattern)
        occurrences_previous = detect_pattern(previous_cells, pattern)
        
        for current_pos in occurrences_current:
            for prev_pos in occurrences_previous:
                if current_pos != prev_pos:  # Check if the pattern has moved
                    replication_count[pattern_name] += 1
    
    return replication_count

def track_occupancy(cells):
    total_cells = cells.size
    live_cells = np.sum(cells)
    occupancy_percentage = (live_cells / total_cells) * 100
    return occupancy_percentage
def simulate_iterations(rows, cols, iterations=1000, max_steps=100):
    all_live_counts = []
    all_pattern_frequencies = []
    all_replication_counts = []
    all_occupancy = []
    
    for _ in range(iterations):
        cells = np.zeros((rows, cols), dtype=int)
        previous_cells = np.zeros((rows, cols), dtype=int)  # Store the previous state
        generate_random(cells)
        
        live_counts = []
        pattern_frequencies = []
        replication_counts = []
        occupancy = []
        
        for step in range(max_steps):
            previous_cells = cells.copy()  # Save the previous state before updating
            cells = update(cells)
            live_count = np.sum(cells)
            live_counts.append(live_count)

            # Track additional metrics
            pattern_frequencies.append(track_pattern_frequency(cells, PATTERNS, step))
            replication_counts.append(track_replication(cells, previous_cells, PATTERNS))
            occupancy.append(track_occupancy(cells))

        all_live_counts.append(live_counts)
        all_pattern_frequencies.append(pattern_frequencies)
        all_replication_counts.append(replication_counts)
        all_occupancy.append(occupancy)

    # Now return these values
    return all_live_counts, all_pattern_frequencies, all_replication_counts, all_occupancy


def plot_pattern_frequencies(all_pattern_frequencies):
    # Plot the frequency of each pattern over time
    for pattern_name in PATTERNS.keys():
        pattern_frequencies = [step_data[pattern_name] for step_data in all_pattern_frequencies]
        plt.plot(pattern_frequencies, label=pattern_name)
    
    plt.xlabel('Iteration')
    plt.ylabel('Pattern Frequency')
    plt.title('Pattern Frequency Over Time')
    plt.legend()
    plt.show()

def plot_replications(all_replication_counts):
    # Plot the replication counts for each pattern
    for pattern_name in PATTERNS.keys():
        replication_counts = [step_data[pattern_name] for step_data in all_replication_counts]
        plt.plot(replication_counts, label=pattern_name)

    plt.xlabel('Iteration')
    plt.ylabel('Replication Count')
    plt.title('Pattern Replication Over Time')
    plt.legend()
    plt.show()

def plot_occupancy(all_occupancy):
    # Plot the occupancy of live cells over time
    plt.plot(all_occupancy, label="Occupancy")
    plt.xlabel('Iteration')
    plt.ylabel('Occupancy (%)')
    plt.title('Occupancy Over Time')
    plt.legend()
    plt.show()
